In [1]:
import mlflow
from prefect import flow, task 
from prefect.tasks import task_input_hash
from datetime import timedelta

In [2]:
name = 'Artificial Data'
experiment = mlflow.get_experiment_by_name(name)
if not experiment:
    experiment_id=mlflow.create_experiment(name)
experiment = mlflow.set_experiment(name)

In [3]:
# Get Experiment Details
print("Experiment_id: {}".format(experiment.experiment_id))
print("Artifact Location: {}".format(experiment.artifact_location))
print("Tags: {}".format(experiment.tags))
print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))

Experiment_id: 4
Artifact Location: file:///home/lpfgarcia/Projects/MLOps/teste/mlruns/4
Tags: {}
Lifecycle_stage: active


In [4]:
run_params = {"experiment_id":experiment.experiment_id,"description":"Testando diferentes classificadores para dados artificiais com prefect e mlflow","tags":{'release.version':'0.0.1'}}

In [5]:
from sklearn.datasets import make_classification

@task
def generate_data(n_samples, n_features):
    return make_classification(n_samples, n_features, n_informative=2, n_redundant=2, random_state=42)

In [6]:
from sklearn.model_selection import train_test_split

@task
def split_data(X, y):
    return train_test_split(X, y, shuffle=False, test_size=int(len(X)*0.1))

In [7]:
@flow
def generate_split(n_samples, n_features):
    X, y = generate_data(n_samples, n_features)
    return split_data(X, y)

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

def classifiers():
    
    clf_list = [
        (KNeighborsClassifier(3), "Nearest Neighbors"),
        (SVC(kernel="linear", C=0.025), "Linear SVM"),
        (DecisionTreeClassifier(max_depth=5), "Decision Tree"),
        (RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1), "Random Forest"),
        (AdaBoostClassifier(), "AdaBoost"),
        (GaussianNB(), "Naive Bayes"),
    ]
    
    return clf_list

In [9]:
@task
def fit(clf, X_train, y_train):
    return clf.fit(X_train, y_train)

In [10]:
@task
def predict(model, X_test):
    return model.predict(X_test)

In [11]:
from sklearn.metrics import accuracy_score

@task
def performance(y_test, y_pred):
    return accuracy_score(y_test, y_pred)

In [12]:
@flow
def evaluate_classifier(clf, X_train, X_test, y_train, y_test):
    model = fit(clf, X_train, y_train)
    y_pred = predict(model, X_test)
    return model, y_pred, performance(y_test, y_pred)

In [13]:
@flow
def evaluate(samples, features):
    
    clf_list = classifiers()
    X_train, X_test, y_train, y_test = generate_split(samples, features)

    for clf, name in clf_list:
    
        with mlflow.start_run() as run:

            mlflow.log_param('name', name)
            mlflow.log_param('samples', samples)  
            mlflow.log_param('features', features)  

            model, y_pred, acc = evaluate_classifier(clf, X_train, X_test, y_train, y_test)

            mlflow.sklearn.log_model(
                sk_model = model,
                artifact_path = name
            )

            mlflow.log_metric('accuracy', acc)

In [14]:
samples = range(10000, 50000, 10000)
features = range(20, 30, 2)

combination = [(x,y) for x in samples for y in features]
    
for samples, features in combination:
    evaluate(samples, features)

11:50:17.903 | INFO    | prefect.engine - Created flow run 'victorious-curassow' for flow 'evaluate'
11:50:18.105 | INFO    | Flow run 'victorious-curassow' - Created subflow run 'elated-curassow' for flow 'generate-split'
11:50:18.164 | INFO    | Flow run 'elated-curassow' - Created task run 'generate_data-eeb4694a-0' for task 'generate_data'
11:50:18.165 | INFO    | Flow run 'elated-curassow' - Executing 'generate_data-eeb4694a-0' immediately...
11:50:18.247 | INFO    | Task run 'generate_data-eeb4694a-0' - Finished in state Completed()
11:50:18.274 | INFO    | Flow run 'elated-curassow' - Created task run 'split_data-b2f518fa-0' for task 'split_data'
11:50:18.274 | INFO    | Flow run 'elated-curassow' - Executing 'split_data-b2f518fa-0' immediately...
11:50:18.339 | INFO    | Task run 'split_data-b2f518fa-0' - Finished in state Completed()
11:50:18.384 | INFO    | Flow run 'elated-curassow' - Finished in state Completed()
11:50:18.515 | INFO    | Flow run 'victorious-curassow' - Cre

11:50:27.033 | INFO    | Flow run 'victorious-curassow' - Finished in state Completed('All states completed.')
11:50:27.094 | INFO    | prefect.engine - Created flow run 'active-ant' for flow 'evaluate'
11:50:27.250 | INFO    | Flow run 'active-ant' - Created subflow run 'invaluable-oxpecker' for flow 'generate-split'
11:50:27.302 | INFO    | Flow run 'invaluable-oxpecker' - Created task run 'generate_data-eeb4694a-0' for task 'generate_data'
11:50:27.303 | INFO    | Flow run 'invaluable-oxpecker' - Executing 'generate_data-eeb4694a-0' immediately...
11:50:27.380 | INFO    | Task run 'generate_data-eeb4694a-0' - Finished in state Completed()
11:50:27.402 | INFO    | Flow run 'invaluable-oxpecker' - Created task run 'split_data-b2f518fa-0' for task 'split_data'
11:50:27.402 | INFO    | Flow run 'invaluable-oxpecker' - Executing 'split_data-b2f518fa-0' immediately...
11:50:27.468 | INFO    | Task run 'split_data-b2f518fa-0' - Finished in state Completed()
11:50:27.511 | INFO    | Flow ru

11:50:35.993 | INFO    | prefect.engine - Created flow run 'misty-deer' for flow 'evaluate'
11:50:36.143 | INFO    | Flow run 'misty-deer' - Created subflow run 'rainbow-earwig' for flow 'generate-split'
11:50:36.199 | INFO    | Flow run 'rainbow-earwig' - Created task run 'generate_data-eeb4694a-0' for task 'generate_data'
11:50:36.199 | INFO    | Flow run 'rainbow-earwig' - Executing 'generate_data-eeb4694a-0' immediately...
11:50:36.280 | INFO    | Task run 'generate_data-eeb4694a-0' - Finished in state Completed()
11:50:36.306 | INFO    | Flow run 'rainbow-earwig' - Created task run 'split_data-b2f518fa-0' for task 'split_data'
11:50:36.307 | INFO    | Flow run 'rainbow-earwig' - Executing 'split_data-b2f518fa-0' immediately...
11:50:36.372 | INFO    | Task run 'split_data-b2f518fa-0' - Finished in state Completed()
11:50:36.420 | INFO    | Flow run 'rainbow-earwig' - Finished in state Completed()
11:50:36.488 | INFO    | Flow run 'misty-deer' - Created subflow run 'swinging-mongoo

11:50:44.963 | INFO    | Flow run 'sandy-snake' - Created subflow run 'electric-cougar' for flow 'generate-split'
11:50:45.018 | INFO    | Flow run 'electric-cougar' - Created task run 'generate_data-eeb4694a-0' for task 'generate_data'
11:50:45.019 | INFO    | Flow run 'electric-cougar' - Executing 'generate_data-eeb4694a-0' immediately...
11:50:45.100 | INFO    | Task run 'generate_data-eeb4694a-0' - Finished in state Completed()
11:50:45.123 | INFO    | Flow run 'electric-cougar' - Created task run 'split_data-b2f518fa-0' for task 'split_data'
11:50:45.123 | INFO    | Flow run 'electric-cougar' - Executing 'split_data-b2f518fa-0' immediately...
11:50:45.192 | INFO    | Task run 'split_data-b2f518fa-0' - Finished in state Completed()
11:50:45.243 | INFO    | Flow run 'electric-cougar' - Finished in state Completed()
11:50:45.313 | INFO    | Flow run 'sandy-snake' - Created subflow run 'fluffy-kiwi' for flow 'evaluate-classifier'
11:50:45.367 | INFO    | Flow run 'fluffy-kiwi' - Creat

11:50:54.074 | INFO    | Flow run 'tough-gorilla' - Executing 'generate_data-eeb4694a-0' immediately...
11:50:54.161 | INFO    | Task run 'generate_data-eeb4694a-0' - Finished in state Completed()
11:50:54.187 | INFO    | Flow run 'tough-gorilla' - Created task run 'split_data-b2f518fa-0' for task 'split_data'
11:50:54.188 | INFO    | Flow run 'tough-gorilla' - Executing 'split_data-b2f518fa-0' immediately...
11:50:54.259 | INFO    | Task run 'split_data-b2f518fa-0' - Finished in state Completed()
11:50:54.313 | INFO    | Flow run 'tough-gorilla' - Finished in state Completed()
11:50:54.389 | INFO    | Flow run 'idealistic-cougar' - Created subflow run 'smooth-bonobo' for flow 'evaluate-classifier'
11:50:54.450 | INFO    | Flow run 'smooth-bonobo' - Created task run 'fit-2157b1fd-0' for task 'fit'
11:50:54.450 | INFO    | Flow run 'smooth-bonobo' - Executing 'fit-2157b1fd-0' immediately...
11:50:54.523 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
11:50:54.549 |

...

11:53:31.837 | INFO    | Task run 'split_data-b2f518fa-0' - Finished in state Completed()
11:53:31.900 | INFO    | Flow run 'enlightened-aardwolf' - Finished in state Completed()
11:53:31.971 | INFO    | Flow run 'wealthy-ibis' - Created subflow run 'famous-moth' for flow 'evaluate-classifier'
11:53:32.034 | INFO    | Flow run 'famous-moth' - Created task run 'fit-2157b1fd-0' for task 'fit'
11:53:32.035 | INFO    | Flow run 'famous-moth' - Executing 'fit-2157b1fd-0' immediately...
11:53:32.144 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
11:53:32.165 | INFO    | Flow run 'famous-moth' - Created task run 'predict-f915ca64-0' for task 'predict'
11:53:32.165 | INFO    | Flow run 'famous-moth' - Executing 'predict-f915ca64-0' immediately...
/home/lpfgarcia/Projects/MLOps/teste/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically prese

11:53:51.754 | INFO    | Flow run 'grinning-agouti' - Finished in state Completed()
11:53:51.824 | INFO    | Flow run 'space-hornet' - Created subflow run 'successful-wildcat' for flow 'evaluate-classifier'
11:53:51.881 | INFO    | Flow run 'successful-wildcat' - Created task run 'fit-2157b1fd-0' for task 'fit'
11:53:51.882 | INFO    | Flow run 'successful-wildcat' - Executing 'fit-2157b1fd-0' immediately...
11:53:51.993 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
11:53:52.015 | INFO    | Flow run 'successful-wildcat' - Created task run 'predict-f915ca64-0' for task 'predict'
11:53:52.016 | INFO    | Flow run 'successful-wildcat' - Executing 'predict-f915ca64-0' immediately...
/home/lpfgarcia/Projects/MLOps/teste/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior 

11:54:12.366 | INFO    | Flow run 'pink-mussel' - Finished in state Completed()
11:54:12.437 | INFO    | Flow run 'logical-walrus' - Created subflow run 'turquoise-dalmatian' for flow 'evaluate-classifier'
11:54:12.495 | INFO    | Flow run 'turquoise-dalmatian' - Created task run 'fit-2157b1fd-0' for task 'fit'
11:54:12.496 | INFO    | Flow run 'turquoise-dalmatian' - Executing 'fit-2157b1fd-0' immediately...
11:54:12.607 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
11:54:12.627 | INFO    | Flow run 'turquoise-dalmatian' - Created task run 'predict-f915ca64-0' for task 'predict'
11:54:12.628 | INFO    | Flow run 'turquoise-dalmatian' - Executing 'predict-f915ca64-0' immediately...
/home/lpfgarcia/Projects/MLOps/teste/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavi

11:54:33.676 | INFO    | Flow run 'monumental-tench' - Finished in state Completed()
11:54:33.756 | INFO    | Flow run 'lumpy-angora' - Created subflow run 'mellow-nuthatch' for flow 'evaluate-classifier'
11:54:33.816 | INFO    | Flow run 'mellow-nuthatch' - Created task run 'fit-2157b1fd-0' for task 'fit'
11:54:33.816 | INFO    | Flow run 'mellow-nuthatch' - Executing 'fit-2157b1fd-0' immediately...
11:54:33.936 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
11:54:33.956 | INFO    | Flow run 'mellow-nuthatch' - Created task run 'predict-f915ca64-0' for task 'predict'
11:54:33.956 | INFO    | Flow run 'mellow-nuthatch' - Executing 'predict-f915ca64-0' immediately...
/home/lpfgarcia/Projects/MLOps/teste/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: t

11:54:56.268 | INFO    | Flow run 'snobbish-mammoth' - Created subflow run 'spiffy-teal' for flow 'evaluate-classifier'
11:54:56.324 | INFO    | Flow run 'spiffy-teal' - Created task run 'fit-2157b1fd-0' for task 'fit'
11:54:56.325 | INFO    | Flow run 'spiffy-teal' - Executing 'fit-2157b1fd-0' immediately...
11:54:56.441 | INFO    | Task run 'fit-2157b1fd-0' - Finished in state Completed()
11:54:56.461 | INFO    | Flow run 'spiffy-teal' - Created task run 'predict-f915ca64-0' for task 'predict'
11:54:56.462 | INFO    | Flow run 'spiffy-teal' - Executing 'predict-f915ca64-0' immediately...
/home/lpfgarcia/Projects/MLOps/teste/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will b